In [ ]:
#importing necessary libraries
import pandas as pd
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import os,glob,nltk
import pickle

#fetching the files from a storage drive
path=os.listdir('bbc')
text=[]
topic=[]
for folder in path:
    print(folder)
    files=glob.glob('bbc/'+folder+'/*.txt')
    for file in files[:400]:
        topic.append(folder)
        data=open(file,'r').read()
        text.append(data)

#displaying number of documents received
print('there are {} documents'.format(len(text)))
print('there are {} topics'.format(len(topic)))

#converting the recieved text to dataframe
df=pd.DataFrame()
text,labels=pd.DataFrame(text,index=None),pd.DataFrame(topic,index=None)
df=pd.concat([text,labels],axis=1)
df.columns=['text','labels']

df.head()

#eleminating the '\n' from the documents
df['text']=df.text.str.replace('\n',' ')
df['text']=df.text.str.lower()

df.head()

#shuffling the data inorder to have a randomness for the model to train
df=shuffle(df)
df.head()
#saving the file
df.to_csv('text.csv',header=True)

df=pd.read_csv('text.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)
df.head()

df.shape

#tranforming the content into vectors
tfidf_vec=TfidfVectorizer()
tfidf_fit=tfidf_vec.fit_transform(df['text'])

tfidf_fit

#normalizes the fitted vectors
tfidf_trans=TfidfTransformer()
tfidf_trans_fit=tfidf_trans.fit_transform(tfidf_fit)

tfidf_trans_fit

#splitting the data into train and test
x_train,x_test,y_train,y_test=train_test_split(tfidf_trans_fit,df['labels'])

#using the naive bayes model
naive_bayes=MultinomialNB()

#training the model
clf=naive_bayes.fit(x_train,y_train)

#predicting the model
naive_bayes.predict(x_test)

#finding the accuracy_score,confusion_matrix and classification report for the predicted and actual data
print('accuracy_score:',accuracy_score(naive_bayes.predict(x_test),y_test))
print('confusion_matrix:',confusion_matrix(naive_bayes.predict(x_test),y_test))
print('classification_report:',classification_report(naive_bayes.predict(x_test),y_test))

#displaying the values in a dataframe format
pd.DataFrame({'actual':y_test,'predicted':naive_bayes.predict(x_test)})

#storing the used vectors,tranformers and models into the local drive using pickle
pickle.dump(tfidf_vec.vocabulary_,open('tfidf_vec_voc.pkl','wb'))
pickle.dump(tfidf_trans,open('tf_idf.pkl','wb'))
pickle.dump(clf,open('naive.pkl','wb'))

#test_data
path=os.listdir('bbc')
text=[]
topic=[]
for folder in path:
    print(folder)
    files=glob.glob('bbc/'+folder+'/*.txt')
    for file in files[400:]:
        topic.append(folder)
        data=open(file,'r').read()
        text.append(data)

print('there are {} documents'.format(len(text)))
print('there are {} topics'.format(len(topic)))

df=pd.DataFrame()
text,labels=pd.DataFrame(text,index=None),pd.DataFrame(topic,index=None)
df=pd.concat([text,labels],axis=1)
df.columns=['text','labels']
df=shuffle(df)

df.to_csv('test.csv')

df.head()

import pandas as pd
df=pd.read_csv('test.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)
df.head()

df.shape

test=df['text']

test=test.str.replace('\n','',)

for doc in test:
    print(doc)

#loading the stored model,vectors and transformers into the notebook
loaded_vec=TfidfVectorizer(vocabulary=pickle.load(open('tfidf_vec_voc.pkl','rb')))
loaded_tfidf=pickle.load(open('tf_idf.pkl','rb'))
loaded_model=pickle.load(open('naive.pkl','rb'))

test_vec=loaded_vec.fit_transform(test)

test_tfidf=loaded_tfidf.fit_transform(test_vec)

predictions=loaded_model.predict(test_tfidf)

print(predictions)

print('accuracy_score:',accuracy_score(df['labels'],predictions))

pd.DataFrame({'actual':df['labels'],'predictions':predictions}).to_csv('predictions.csv')

pd.read_csv('predictions.csv')

'''Over using the naive bayes model we have the good predictions with more than 90+ accuracy rate'''